# Diffusion Equation (2D)

## Introduction

Assuming constant thermal parameters and considering only radiogenic heat production, the 2D heat conduction equation is given by:

$$
\frac{\partial{T}}{\partial{t}} = \kappa \left( \frac{\partial^2{T}}{\partial{x^2}} + \frac{\partial^2{T}}{\partial{y^2}} \right) + \frac{Q}{\rho c_p}, \tag{1}
$$

where $\rho$ is the density [kg/m<sup>3</sup>], $c_p$ is the specific heat capacity [J/kg/K], $\kappa = k/(\rho c_p)$ is the thermal diffusivity [m<sup>2</sup>/s], and $Q$ is the volumetric radiogenic heat production [W/m<sup>3</sup>].  

This equation describes the temporal evolution of temperature under the assumption that heat transport occurs purely by diffusion.


## The Problem

We consider equation (1) in a 2D rectangular domain with a width of $L = 200$ km and a depth of $H = 100$ km (Fig. 1). The surface temperature is fixed at 0 °C, and within the domain the temperature increases linearly according to a prescribed thermal gradient. The lateral boundary conditions can be chosen as either *Dirichlet* or *Neumann*. At the base of the domain, a region in the center with width $W_{Plume}$ is assigned a higher temperature than the surrounding material.  

<img src="../Figures/Exercise05a_1.png" alt="drawing" width="450"/> <br>  
**Fig. 1.** Model setup  

This configuration represents the situation of a plume head positioned beneath the lithosphere. Over time, the excess temperature of the plume heats the overlying lithosphere.  

To solve this problem with Julia, we first need to load the required modules (`ExtendableSparse`, `Plots`) and the submodule (`GeoModBox.HeatEquation.TwoD`).

In [ ]:
using Plots, GeoModBox.HeatEquation.TwoD, ExtendableSparse
start = time()

We now define the model dimensions ($L$, $H$) and physical parameters ($k$, $Q$), as well as the discretization scheme (explicit or implicit):

In [ ]:
FDSchema    = :explicit
# Physical parameters --------------------------------------------------- #
P      = (
    L       =   200e3,          # Length of the model [m]
    H       =   100e3,          # Height of the model [m]
    k       =   6,              # Thermal conductivity [W·m⁻¹·K⁻¹]
    cp      =   1000,           # Heat capacity [J·kg⁻¹·K⁻¹]
    ρ       =   3200,           # Density [kg·m⁻³]
    K0      =   273.15,         # Kelvin at 0 °C
    Q0      =   0,              # Background heat production rate [W·m⁻³]
)
P1      =   (
    κ       =   P.k / P.ρ / P.cp,   # Thermal diffusivity [m²·s⁻¹]
    Tbot    =   1300.0 + P.K0,      # Temperature at the bottom boundary [K]
    Ttop    =   0.0 + P.K0,         # Temperature at the top boundary [K]
    Tplume  =   2000.0 + P.K0,      # Temperature of the plume [K]
    Xplume  =   P.L/2.0,            # x-coordinate of the plume center [m]
    Wplume  =   50e3                # Width of the plume [m]
)
P   = merge(P,P1)
# ----------------------------------------------------------------------- #


Next, we define the number of grid points and the grid spacing:

In [ ]:
# Numerical parameters -------------------------------------------------- #
NC      = (
    x       =   100,             # Grid points in the x-direction
    y       =   50,              # Grid points in the y-direction
)
Δ       = (
    x       = P.L/NC.x,          # Grid spacing in the x-direction [m]
    y       = P.H/NC.y           # Grid spacing in the y-direction [m]
)
# ----------------------------------------------------------------------- #


With these parameters, the numerical grid can be defined:

In [ ]:
# Grid generation ------------------------------------------------------- #
x       = (
    c       =   LinRange(0.0 + Δ.x/2.0, P.L - Δ.x/2.0, NC.x),
)
y       = (
    c       =   LinRange(-P.H + Δ.y/2.0, 0.0 - Δ.y/2.0, NC.y),
)
# ----------------------------------------------------------------------- #

Since we are dealing with a time-dependent problem, we next define the parameters for the time integration. Note that in the case of the explicit finite-difference scheme, the diffusion stability criterion must be satisfied.

In [ ]:
# Time parameters ------------------------------------------------------- #
T       = (
    dn      =   100,            # Increment for graphical output,
                                # i.e., here only every 25th time step
    year    =   365.25*3600*24, # Seconds per year    
    dtfac   =   0.9,            # Safety factor for time step size
    Δ       =   [0.0],          # Time step length [s]
    nt      =   [0]             # Number of time steps (initialized)
)
T1      = (
    tmax    =   200 * 1e6 * T.year,    # Maximum simulation time [s]
    Δ       =   T.dtfac * (1.0 / (2.0 * P.κ * (1.0/Δ.x^2 + 1/Δ.y^2)))
)
T           = merge(T,T1)

nt          = floor(Int, T.tmax/T.Δ)  # Total number of time steps
Time        = zeros(nt)
# ----------------------------------------------------------------------- #

To visualize the results as an animated GIF, we also need to specify the file location and name:

In [ ]:
# Animationssettings ---------------------------------------------------- #
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("05_2D_Plume_",FDSchema)
save_fig    =   1
# ----------------------------------------------------------------------- #

Next, we define the initial conditions and data arrays for our problem:


In [ ]:
# Initial temperature field --------------------------------------------- #
D       =   (
    Q           =   zeros(NC...),
    T           =   zeros(NC...),
    T_ex        =   zeros(NC.x+2, NC.y+2),
    Tmax        =   zeros(nt),
    Tprofile    =   zeros(NC.y, nt),
    ρ           =   zeros(NC...),
)
# Background field for heat sources
D.Q     .=  P.Q0
D.ρ     .=  P.ρ
        
# Lithosphere temperature – linearly increasing with depth
for i = 1:NC.x, j = 1:NC.y
    D.T[i,j]     =   P.Ttop + abs(y.c[j]/P.H) * P.Tbot
    if abs(x.c[i] - P.Xplume) <= P.Wplume/2.0 && j == 1
        D.T[i,j]    =   P.Tplume
    end
end
D.T_ex[2:end-1,2:end-1]     .=  D.T

# Visualize initial condition ------------------------------------------- #
p = heatmap(x.c ./ 1e3, y.c ./ 1e3, (D.T .- P.K0)', 
        color=:viridis, colorbar=true, aspect_ratio=:equal, 
        xlabel="x [km]", ylabel="z [km]", 
        title="Temperature", 
        xlims=(0, P.L/1e3), ylims=(-P.H/1e3, 0.0), 
        clims=(0, 2000))

contour!(p, x.c./1e3, y.c/1e3, D.T' .- P.K0, levels=:10, linecolor=:black)
if save_fig == 0
    display(p)
end
# ----------------------------------------------------------------------- #


## The Solution

### Discretization

To solve the problem numerically, the model domain is subdivided into a computational grid. We assume that the temperature and all thermal parameters are defined at *cell-centered* grid points (see Fig. 2). This setup corresponds to the discretization used in the stationary problem.  

As before, we use *ghost nodes* in the grid to correctly incorporate the boundary conditions.  

#### Grid and Indexing

<img src="../Figures/Exercise04_2.png" alt="drawing" width="450"/> <br>  
**Fig. 2.** Staggered grid.  

The staggered grid enables a *conservative* finite-difference approximation. Here, the heat flux  
$q_{i,j} = -k \frac{\partial{T}}{\partial{x_{i,j}}}$ is defined at the midpoints of the grid lines, while the temperature is defined at the cell centers. (Strictly speaking, the thermal conductivity is also defined on the grid lines; since it is constant, this can be ignored. For variable thermal parameters, the discretization must be adjusted.)  

With cell-centered temperatures combined with *ghost nodes*, boundary conditions can be implemented in a straightforward way and with the same order of accuracy as the central difference quotients in the interior of the model.  

When indexing grid points, we distinguish between *local* and *global* indices:  
- The local index describes the position on the $(i,j)$ grid.  
- The global index is a running index from 1 to $n_x \times n_y$, corresponding to the number of equations, i.e., the total number of interior grid points. The global index is also used when assembling the coefficient matrix of the linear system.  

For each grid point (i.e., each equation), a numerical *stencil* specifies the neighboring grid points relevant for that equation. The coefficients for these stencil points are nonzero, while all others are zero. The stencil notation often follows a compass scheme: South, West, Center, East, and North. For each equation, the global index ($ii$) of the stencil points is given by their relative position to the center of the stencil:  

$$
iS = ii - nx,\\\\
iW = ii - 1,\\\\   
iZ = ii, \\\\
iE = ii + 1,\\\\
iN = ii + nx.
$$

#### Finite-Difference Approximation

The partial differential equation can now be approximated by finite differences:  

**Explicit (Forward Euler scheme)**  
$$
T_{i,j}^{n+1} = T_{i,j}^n + \kappa \Delta{t} \left(\frac{T_{i-1,j}^{n} - 2 T_{i,j}^{n} + T_{i+1,j}^{n}}{\Delta{x^2}} \right) + \kappa \Delta{t} \left(\frac{T_{i,j-1}^{n} - 2 T_{i,j}^{n} + T_{i,j+1}^{n}}{\Delta{y^2}} \right) + \frac{Q_{i,j} \Delta{t}}{\rho c_p}, \tag{2}
$$  

where $n+1$ and $n$ denote the new and current time steps.  

**Implicit (Backward Euler scheme)**  
$$
-b T_{i,j-1}^{n+1} - a T_{i-1,j}^{n+1} + (2a + 2b + c) T_{i,j}^{n+1} - a T_{i+1,j}^{n+1} - b T_{i,j+1}^{n+1} = T_{i,j}^{n} + \frac{Q_{i,j}}{\rho c_p}, \tag{3}
$$  

where $a = \kappa / \Delta{x^2}$, $b = \kappa / \Delta{y^2}$, and $c = 1 / \Delta{t}$.  
Thus, the system can be written as a linear system with a coefficient matrix $\mathbf{A}$ containing five nonzero diagonals, an unknown vector $T^{n+1}$, and a known vector $T^n$.  

### Boundary Conditions

The temperature at the *ghost nodes* is defined for *Dirichlet* and *Neumann* boundary conditions in the same way as in the [stationary 2D case](./04_2D_Diffusion_Stationary.ipynb) and the corresponding [explicit](./02_1D_Heat_explicit.ipynb) and [implicit](./03_1D_Heat_implicit.ipynb) 1D versions.  

- In the **explicit scheme**, the temperatures at the *ghost nodes* are directly used in the update equations for the adjacent interior points.  
- In the **implicit scheme**, the problem is again formulated as a linear system. In this case, the coefficients and right-hand side of the equations for the **interior points near the boundaries** must be modified according to the boundary conditions, making use of the *ghost node* values (derivation in the lecture).  

When initializing the boundary conditions, a small trick is used: the temperature at the *ghost nodes* is defined later in the script.


In [ ]:
# Boundary Conditions --------------------------------------------------- #
BC      =   (type    = (W=:Neumann, E=:Neumann, N=:Dirichlet, S=:Dirichlet),
           val     = (W=zeros(NC.y),E=zeros(NC.y),N=D.T[:,end],S=D.T[:,1]))
#BC     = (type    = (W=:Dirichlet, E=:Dirichlet, N=:Dirichlet, S=:Dirichlet),
#             val     = (W=D.T[1,:],E=D.T[end,:],N=D.T[:,end],S=D.T[:,1]))
# ----------------------------------------------------------------------- #

### Solving the Problem

#### Linear System

For the implicit case, we now initialize the coefficient matrix and the right-hand side:

In [ ]:
# Linear System of Equations -------------------------------------------- #
Num     =   (T=reshape(1:NC.x*NC.y, NC.x, NC.y),)
ndof    =   maximum(Num.T)
K       =   ExtendableSparseMatrix(ndof,ndof)
rhs     =   zeros(ndof)
# ----------------------------------------------------------------------- #

#### Time Loop

Numerically, we can now solve the PDE within a time loop in different ways.  

- For the **explicit scheme**, we will first program the equations and boundary conditions manually before making use of the built-in functions from the `GeoModBox.HeatEquation.TwoD` submodule.  
- For the **implicit scheme**, we directly apply the built-in function `BackwardEuler2Dc!()`.  


In [ ]:
# Time loop ------------------------------------------------------------- #
for n = 1:nt
    println(n)
    # Store the temperature profile at x = L/2
    @. D.Tprofile[:,n]  =  (D.T[convert(Int,NC.x/2),:] 
                                + D.T[convert(Int,NC.x/2)+1,:]) / 2
    D.Tmax[n]        =   maximum((D.T[:,convert(Int,NC.y/2)] 
                                + D.T[:,convert(Int,NC.y/2)+1]) / 2)

    if n > 1
        if FDSchema == :explicit             
            ForwardEuler2Dc!(D, P.κ, Δ.x, Δ.y, T.Δ, D.ρ, P.cp, NC, BC)
        elseif FDSchema == :implicit
            BackwardEuler2Dc!(D, P.κ, Δ.x, Δ.y, T.Δ, D.ρ, P.cp, NC, BC, rhs, K, Num)
        end                            
        Time[n]     =   Time[n-1] + T.Δ
    end    
    
    if mod(n, T.dn) == 0 || n == 1 || n == nt
        p = heatmap(x.c ./ 1e3, y.c ./ 1e3, (D.T.-P.K0)', 
        color=:viridis, colorbar=true, aspect_ratio=:equal, 
        xlabel="x [km]", ylabel="z [km]", 
        title="Temperature", 
        xlims=(0, P.L/1e3), ylims=(-P.H/1e3, 0.0), 
        clims=(0, 2000))

        contour!(p, x.c./1e3, y.c/1e3, D.T' .- P.K0, levels=:10, linecolor=:black)
        if save_fig == 1
            Plots.frame(anim)
        else
            display(p)                        
        end
    end
end


## Results

For clearer presentation and analysis, we plot a vertical temperature profile as a function of depth, as well as the maximum of a horizontal temperature profile as a function of time at the center of the domain:


In [ ]:
q = plot(D.Tprofile[:,1:T.dn:end].-P.K0,y.c./1e3,
        label="",xlabel="T_{x=L/2}",ylabel="Depth [km]",
        title="Temperature profile",
        layout=(1,2),subplot=1)

plot!(q,Time./T.year/1e6,D.Tmax.-P.K0,
        label="",xlabel="Time [My]",ylabel="T_{max} [°C]",
        subplot=2)


Next, we create and save the animation:

In [ ]:
if save_fig == 1
        # Write the frames to a GIF file
        Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)        
        savefig(string("./Results/05_Plume_TProfile_Tmax_",FDSchema,".png"))
else
    display(q)
end
foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))
stop=time()
println(stop-start)